In [3]:
import numpy as np
import xarray as xr
import pickle as pk
import pandas as pd
from matplotlib import pyplot as plt
from math import erfc
import math
from scipy.optimize import curve_fit as curve_fit

In [12]:
path = '/Volumes/eSSD0/Papers/JoC_gFBK/Data/SI/'

sino = pk.load(open(path+'b.e12.SI.T31_g37.039rad.ctl.2_ctl_diag.pk','rb'))
pic = pk.load(open(path+'b.e12.pi-control.T31_g37.ctl.2_ctl_diag.pk','rb'))
pic_tas = pic['tas'][-1]
sino_tas_gam = sino['tas']
sino_tsi_gam = sino['TSI']
sino_dtas_gam = sino_tas_gam-pic_tas

In [13]:
def fit_sin(tt, yy):
    '''Fit sin to the input time sequence'''
    tt = np.array(tt)
    yy = np.array(yy)
    ff = np.fft.fftfreq(len(tt), (tt[1]-tt[0]))
    Fyy = abs(np.fft.fft(yy))
    guess_freq = abs(ff[np.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = np.std(yy) * 2.**0.5
    guess_offset = np.mean(yy)
    guess = np.array([guess_amp, 2.*np.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * np.sin(w*t + p) + c
    popt, pcov = curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*np.pi)
    fitfunc = lambda t: A * np.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": np.max(pcov), "rawres": (guess,popt,pcov)}

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def ki(mag,lag,pm,crossfreq):
    lag_r = math.radians(lag)
    pm_r = math.radians(pm)
    return (crossfreq/mag)/(np.sqrt(1+(np.tan(pm_r-(np.pi/2)-lag_r)**2)))

def kp (ki,lag,pm,crossfreq):
    lag_r = math.radians(lag)
    pm_r = math.radians(pm)
    return (ki/crossfreq)*np.tan(pm_r-(np.pi/2)-lag_r)

In [14]:
print(f'Angular Frequency = 0.39 radians\n')
ts_fit = fit_sin(np.arange(len(sino_tas_gam)),sino_dtas_gam)
ts_sin = ts_fit['fitfunc'](np.arange(len(sino_tas_gam)))
print('Tas data specs')
print( "Amplitude=%(amp)s\nAngular freq.=%(omega)s\nphase=%(phase)s\nfreq=%(freq)s\nperiod=%(period)s\noffset=%(offset)s" % ts_fit,'\n')

Angular Frequency = 0.39 radians

Tas data specs
Amplitude=0.3650798708892348
Angular freq.=0.3921951606698162
phase=-0.03665504982346449
freq=0.06241979847732135
period=16.020557970294067
offset=-0.020869992164110115 



In [15]:
print(f'Angular Frequency = 0.39 radians\n')
tsi_fit = fit_sin(np.arange(len(sino_tsi_gam)),sino_tsi_gam)
tsi_sin = ts_fit['fitfunc'](np.arange(len(sino_tsi_gam)))
print('Tas data specs')
print( "Amplitude=%(amp)s\nAngular freq.=%(omega)s\nphase=%(phase)s\nfreq=%(freq)s\nperiod=%(period)s\noffset=%(offset)s" % tsi_fit,'\n')

Angular Frequency = 0.39 radians

Tas data specs
Amplitude=13.350906677798855
Angular freq.=0.3926990816987241
phase=0.40971017105919993
freq=0.06249999999999999
period=16.000000000000004
offset=1360.89 



In [17]:
Gs_mag = (ts_fit['amp']/tsi_fit['amp'])*tsi_fit['amp'] #convert from W/m2 to % SOL
Gs_phase = (ts_fit['phase']-tsi_fit['phase'])*(180/np.pi)
print(f'Amplitude Gain - {round(Gs_mag,2)} K/%\nPhase lag - {round(Gs_phase,2)} degrees\n')

Amplitude Gain - 0.37 K/%
Phase lag - -25.57 degrees



In [25]:
#sample values from bens papers
wi_s = 0.2
mag_s = 0.447
phase_s = 33
pm_s = 84

gs_i = ki(mag_s,-phase_s,pm_s,wi_s)
gs_k = kp(gs_i,-phase_s,pm_s,wi_s)

print(f'Ki = {gs_i}\nKp = {gs_k}\nPhase Margin = {pm_s} degrees')

Ki = 0.3986606372207463
Kp = 1.015638701878181
Phase Margin = 84 degrees


In [18]:
#phase margin
pm = 60
wi = tsi_fit['omega']
lag = np.abs(Gs_phase)+ math.degrees(wi)
gi = ki(Gs_mag,-lag,pm,wi)
gk = kp(gi,-lag,pm,wi)
print(f'Ki = {gi}\nKp = {gk}\nPhase Margin = {pm} degrees')
print(f'Total Phase Lag = {int(lag)} degrees\n')


Ki = 1.0225712673283145
Kp = 0.8498388537684161
Phase Margin = 60 degrees
Total Phase Lag = 48 degrees



In [26]:
math.degrees(0.2)

11.459155902616466

In [22]:
lag

48.074843278001424

In [23]:
np.abs(Gs_phase)

25.57484327800143

In [24]:
lag

48.074843278001424